# MLZoomcamp 2022 - Session #8 - Homework

Author: José Victor

* Dataset: [Dino or Dragon?](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon)

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

## Imports

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras import layers

In [2]:
tf.config.list_physical_devices('GPU')

[]

## Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and validation.

## Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(150, 150, 3)`
* Next, create a convolutional layer (`Conv2D`)
    * Use 32 filters
    * Kernel size should be `(3, 3)` (that's the size of the filter)
    * Use `relu` as activation
* Reduce the size of the feature map with max pooling (`MaxPooling2D`)
    * Set the pooling size to `(2, 2)`
* Turn the multi-dimensional result into vectors using a `Flatten` layer
* Next, add a `Dense` layer with 64 neurons and `relu` activation
* Finally, create the `Dense` layer with 1 neuron - this will be the output
    * The output layer should have an activation - use the appropriate activation for the binary classification case

As optimizer use `SGD` with the following parameters:

* `SGD(lr=0.002, momentum=0.8)`

For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [3]:
model = keras.Sequential([layers.Conv2D(32, kernel_size=(3, 3), activation="relu", input_shape=(150, 150, 3)),
                          layers.MaxPooling2D(pool_size=(2, 2)),
                          layers.Flatten(),
                          layers.Dense(64, activation="relu"),
                          layers.Dense(1, activation="sigmoid")],
                         name="CNNHomework")

In [4]:
optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)

e:\vs_code\mlzoomcamp2022_jvscursulim\env\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## Question 1

Since we have a binary classification problem, what is the best loss function for us?

* (X) `binary crossentropy`
* ( ) `focal loss`
* ( ) `mean squared error`
* ( ) `categorical crossentropy`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

In [5]:
loss = keras.losses.BinaryCrossentropy()

## Question 2

What's the total number of parameters of the model? You can use the `summary` method for that.

* ( ) 9215873
* (X) 11215873
* ( ) 14215873
* ( ) 19215873

In [6]:
model.summary()

Model: "CNNHomework"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 175232)            0         
                                                                 
 dense (Dense)               (None, 64)                11214912  
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
___________________________________________

## Generators and Training

For the next two questions, use the following data generator for both train and validation:

```python
ImageDataGenerator(rescale=1./255)
```

* We don't need to do any additional pre-processing for the images.
* When reading the data from train/val directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and validation

For training use `.fit()` with the following params:

```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)
```

In [7]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory("../data/train",
                                         target_size=(150,150),
                                         batch_size=20,
                                         shuffle=True,
                                         class_mode="binary")

Found 1594 images belonging to 2 classes.


In [8]:
train_generator.class_mode

'binary'

In [9]:
validation_datagen = ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory("../data/test",
                                         target_size=(150,150),
                                         batch_size=20,
                                         shuffle=True,
                                         class_mode="binary")

Found 394 images belonging to 2 classes.


In [10]:
validation_generator.class_mode

'binary'

In [11]:
model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [12]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
80/80 [==============================] - 20s 238ms/step - loss: 0.6232 - accuracy: 0.6506 - val_loss: 0.5230 - val_accuracy: 0.7817
Epoch 2/10
80/80 [==============================] - 11s 139ms/step - loss: 0.4920 - accuracy: 0.7804 - val_loss: 0.4461 - val_accuracy: 0.8020
Epoch 3/10
80/80 [==============================] - 11s 138ms/step - loss: 0.4220 - accuracy: 0.8168 - val_loss: 0.3841 - val_accuracy: 0.8376
Epoch 4/10
80/80 [==============================] - 11s 139ms/step - loss: 0.3734 - accuracy: 0.8476 - val_loss: 0.3687 - val_accuracy: 0.8401
Epoch 5/10
80/80 [==============================] - 11s 139ms/step - loss: 0.3434 - accuracy: 0.8626 - val_loss: 0.3339 - val_accuracy: 0.8579
Epoch 6/10
80/80 [==============================] - 11s 139ms/step - loss: 0.3039 - accuracy: 0.8777 - val_loss: 0.3371 - val_accuracy: 0.8503
Epoch 7/10
80/80 [==============================] - 11s 139ms/step - loss: 0.2647 - accuracy: 0.8996 - val_loss: 0.3768 - val_accuracy: 0.8249

## Question 3

What is the median of training accuracy for all the epochs for this model?

* ( ) 0.40
* ( ) 0.60
* (X) 0.90
* ( ) 0.20

In [13]:
np.median(model.history.history["accuracy"])

0.870138019323349

## Question 4

What is the standard deviation of training loss for all the epochs for this model?

* (X) 0.11
* ( ) 0.66
* ( ) 0.99
* ( ) 0.33

In [14]:
np.std(model.history.history["loss"])

0.12514084418783533

## Data Augmentation

For the next two questions, we'll generate more data using data augmentations.

Add the following augmentations to your training data generator:

* `rotation_range=40`
* `width_shift_range=0.2`
* `height_shift_range=0.2`
* `shear_range=0.2`
* `zoom_range=0.2`
* `horizontal_flip=True`
* `fill_mode='nearest'`

In [16]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

train_generator = train_datagen.flow_from_directory("../data/train",
                                         target_size=(150,150),
                                         batch_size=20,
                                         shuffle=True,
                                         class_mode="binary")

Found 1594 images belonging to 2 classes.


## Question 5

Let's train our model for 10 more epochs using the same code as previously. Make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of validation loss for all the epochs for the model trained with augmentations?

* ( ) 0.15
* ( ) 0.77
* (X) 0.37
* ( ) 0.97

In [17]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator
)

Epoch 1/10
80/80 [==============================] - 16s 195ms/step - loss: 0.4548 - accuracy: 0.7880 - val_loss: 0.3463 - val_accuracy: 0.8376
Epoch 2/10
80/80 [==============================] - 16s 197ms/step - loss: 0.4215 - accuracy: 0.8105 - val_loss: 0.7727 - val_accuracy: 0.6269
Epoch 3/10
80/80 [==============================] - 16s 197ms/step - loss: 0.4239 - accuracy: 0.8005 - val_loss: 0.2961 - val_accuracy: 0.8680
Epoch 4/10
80/80 [==============================] - 16s 198ms/step - loss: 0.4180 - accuracy: 0.8130 - val_loss: 0.3813 - val_accuracy: 0.8350
Epoch 5/10
80/80 [==============================] - 16s 199ms/step - loss: 0.3771 - accuracy: 0.8294 - val_loss: 0.5419 - val_accuracy: 0.7690
Epoch 6/10
80/80 [==============================] - 16s 198ms/step - loss: 0.3811 - accuracy: 0.8325 - val_loss: 0.4725 - val_accuracy: 0.7893
Epoch 7/10
80/80 [==============================] - 16s 203ms/step - loss: 0.3524 - accuracy: 0.8507 - val_loss: 0.4656 - val_accuracy: 0.8071

In [18]:
np.mean(model.history.history["val_loss"])

0.454131880402565

## Question 6 

What's the average of validation accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* (X) 0.84
* ( ) 0.54
* ( ) 0.44
* ( ) 0.24

In [19]:
np.mean(model.history.history["val_accuracy"][5:])

0.810152280330658